In [3]:
import os
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
import xgboost as xgb
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import fetch_openml

import matplotlib.pyplot as plt

from helpers.datasetHelper import get_samples, split_healthy_data
from MyPSO import MyPSO

print("importing done")



importing done


In [8]:
def load_methylation_data(file_path):
    """Load DNA methylation data from a file"""
    # Read the CSV file
    data = pd.read_csv(file_path)
    
    # Extract feature names, exclude first column (sample names) and last column (target)
    feature_names = np.array(data.columns[1:-1])
    
    # Extract data: X contains all columns except the first (sample names) and last (target)
    X = data.iloc[:, 1:-1].values
    
    # Extract target: last column
    y = data.iloc[:, -1].values
    
    return X, y, feature_names



In [13]:
directory_path = './datasets'
data_path = os.path.join(directory_path, 'DT.Healthy.csv')

"""Main function to run the PSO feature selection with XGBoost"""
# Load methylation data
print("Loading DNA methylation data...")

data_health = get_samples(data_path)

# Load the PAN-CANCER-TRANSPOSED.csv data
healthy_cases, prebrca_cases, cancer_cases = split_healthy_data(data_health)

# Combine the data into a single dataframe
# Tag each list of cases
healthy_cases = pd.DataFrame(healthy_cases)
healthy_cases['Tag'] = 'HEALTHY'
prebrca_cases = pd.DataFrame(prebrca_cases)
prebrca_cases['Tag'] = 'PRE-BRCA'
cancer_cases = pd.DataFrame(cancer_cases)
cancer_cases['Tag'] = 'BRCA'

df_cancer = pd.concat([healthy_cases, prebrca_cases, cancer_cases], ignore_index=True) #blood samples
# The last column is the target classes
# Ensure all data is numeric
X = df_cancer.iloc[:, :-1].apply(pd.to_numeric, errors='coerce')
Y = df_cancer.iloc[:, -1]
# Label encode the target variable
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(Y)


Loading DNA methylation data...


c:\Leandro\Master\Repository\Mamo.IA\helpers\datasetHelper.py:6: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,

In [14]:
# Fill missing values with the lowest value of its cpg site
X = X.apply(lambda col: col.fillna(col.min()), axis=0)



In [ ]:
print(f"Label encoding mapping: {dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))}")
feature_names = {index: value for index, value in enumerate(data_health)}

n_features = X.shape[1]
print(f"Loaded dataset with {n_features} features and {len(Y)} samples")

# Initialize PSO
pso = MyPSO(n_particles=30, 
            n_features=n_features, 
            max_iter=100, 
            init_prob=0.3,
            min_features=3)


Label encoding mapping: {'BRCA': np.int64(0), 'HEALTHY': np.int64(1), 'PRE-BRCA': np.int64(2)}
Loaded dataset with 27578 features and 790 samples


In [ ]:
# Run optimization
# Run optimization
best_position, best_fitness = pso.optimize(X, Y)

# Ensure the length of best_position matches the number of features
if len(best_position) != n_features:
	raise ValueError(f"Length of best_position ({len(best_position)}) does not match the number of features ({n_features}).")


  0%|          | 0/100 [00:00<?, ?it/s]


IndexError: Boolean index has wrong length: 100 instead of 27578